In [ ]:
import os

os.environ["CloudStorage"] = "allomanusiayayaya"

os.environ["GoogleProject"] = "qwiklabs-gcp-03-b58c1940e6a4"

os.environ["GoogleRegion"] = "us-central1"

os.environ["TensorFlowVersion"] = "2.6"

os.environ["PythonVersion"] = "3.7"

In [ ]:
%%bash

export project=$(gcloud config list project --format "value(core.project)")

In [ ]:
project

In [ ]:
%%bash

bq --location=US mk --dataset --description "Baby Weight" $project:babyweight

In [ ]:
%%bash

bq ls

In [ ]:
%%bash

gsutil mb -l ${GoogleRegion} gs://${CloudStorage}

In [ ]:
%%bash

gsutil ls

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE
  babyweight.babyweight_data_
SELECT
  weight_pounds,
  CAST(is_male AS STRING) AS is_male,
  mother_age,
  CASE
    WHEN plurality = 1 THEN "Single(1)"
    WHEN plurality = 2 THEN "Twins(2)"
    WHEN plurality = 3 THEN "Triplets(3)"
    WHEN plurality = 4 THEN "Quadruplets(4)"
    WHEN plurality = 5 THEN "Quintuplets(5)"
  END AS plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year = 2010
  AND weight_pounds > 0
  AND mother_age > 0
  AND plurality > 0
  AND gestation_weeks > 0

In [ ]:
import google.cloud.bigquery as bigquery

client = bigquery.Client()

dataset = "babyweight"

dataref = client.dataset(dataset_id=dataset, project=client.project)

for label in ["train", "testi"]:

  destination_url = os.path.join("gs://", bucket, dataset, "data", "{}*.csv".format(label))

  table = "babyweight_data_{}".format(label)

  tableref = dataref.table(table)

  extract = client.extract_table(tableref, destination_url, location="US")

  extract.result()